<h1>Đồ án cuối kì - Nhập môn khoa học dữ liệu<h1>
<h2>Tiền xử lý và khám phá dữ liệu<h2>

Khai báo thư viện

In [52]:
import pandas as pd
import numpy as np
import pycountry_convert as pc

Đọc dữ liệu, bỏ cột Cause type

In [53]:
df = pd.read_csv('raw_data.csv')
df = df.drop(columns='Cause type')
df.head()

,Cause death,Age,Year,Country,Number of deaths
0,HIV/AIDS,0,2020,Albania,NaN
1,HIV/AIDS,0,2020,Antigua and Barbuda,NaN
2,HIV/AIDS,0,2020,Argentina,NaN
3,HIV/AIDS,0,2020,Armenia,NaN
4,HIV/AIDS,0,2020,Australia,0.0


Chia 'Number of deaths' theo 'Cause death' và gom nhóm theo 'Country', 'Year' và 'Age'

In [54]:
cause_death = df['Cause death'].unique() # danh sách cause death
df = pd.pivot_table(df, values='Number of deaths', index=['Country', 'Year', 'Age'], columns=['Cause death'], aggfunc=np.sum).reset_index()
df = df.rename_axis(None, axis=1)
display(df.head())
print(f'{df.shape[0]} rows x {df.shape[1]} columns')
print(f'Nguyên nhân tử vong ({len(cause_death)}):')
print(*cause_death, sep='\n')

,Country,Year,Age,Brain and nervous system cancers,Covid-19,Dengue,Drownings,Fires,HIV/AIDS,Leukaemia,Malaria,Natural disasters,Road traffic accidents,Self-inflicted injuries,Tuberculosis,Violence,War
0,Albania,2000,0,0.0,NaN,NaN,1.0,1.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,Albania,2000,1-4,0.0,NaN,NaN,12.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,Albania,2000,15-24,4.0,NaN,NaN,4.0,2.0,0.0,10.0,0.0,3.0,6.0,11.0,0.0,29.0,0.0
3,Albania,2000,25-34,6.0,NaN,NaN,5.0,0.0,0.0,9.0,0.0,1.0,13.0,20.0,0.0,34.0,0.0
4,Albania,2000,35-54,33.0,NaN,NaN,4.0,1.0,0.0,18.0,0.0,1.0,15.0,9.0,4.0,35.0,0.0


18984 rows x 17 columns
Nguyên nhân tử vong (14):
HIV/AIDS
Malaria
Tuberculosis
Dengue
Covid-19
Brain and nervous system cancers
Leukaemia
Road traffic accidents
Fires
Drownings
Natural disasters
Self-inflicted injuries
Violence
War


Đếm số dòng có giá trị 0 hoặc NaN của các cột 'Country', 'Year', 'Age'

In [55]:
print('Country: ', ((df.Country == 0) | (df.Country.isnull())).sum())
print('Year   : ', ((df.Year == 0) | (df.Year.isnull())).sum())
print('Age    : ', ((df.Age == 0) | (df.Age.isnull())).sum())

Country:  0
Year   :  0
Age    :  0


Như vậy, chỉ có những cột nguyên nhân tử vong có chứa giá trị 0 hoặc NaN. </br>
Một dòng mà dữ liệu chỉ chứa giá trị 0 hoặc NaN thì dòng đó không có ý nghĩa.</br>
==> Loại bỏ những dòng này.

In [56]:
print('Kích thước trước khi xoá: ', df.shape)
df = df[((df == 0) | (df.isnull())).sum(1) < len(cause_death)]
df.reset_index(drop=True, inplace=True)
print('Kích thước sau khi xoá  : ', df.shape)


Kích thước trước khi xoá:  (18984, 17)
Kích thước sau khi xoá  :  (14928, 17)


Thêm cột tên khu vực của quốc gia

In [57]:
# các quốc gia không được hỗ trợ bởi thư viện
exceptive_countries = {
    'China, Hong Kong SAR': 'Asia',
    'Iran (Islamic Republic of)': 'Asia',
    'Republic of Korea': 'Asia'
}

def country_to_continent(country_name):
    if country_name in exceptive_countries:
        return exceptive_countries[country_name]

    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

df.insert(loc = 0, column = 'Continent', value = df.Country.apply(country_to_continent))

continents = df.Continent.unique() # danh sách các khu vực
print(f'Các khu vực ({len(continents)}):')
print(*continents, sep='\n')

Các khu vực (6):
Europe
North America
South America
Asia
Oceania
Africa


Chuyển kiểu dữ liệu của 'Year' sang datetime và kiểm tra kiểu dữ liệu của mỗi cột

In [58]:
df.Year = pd.to_datetime(df.Year, format='%Y')
df.dtypes 

Continent                                   object
Country                                     object
Year                                datetime64[ns]
Age                                         object
Brain and nervous system cancers           float64
Covid-19                                   float64
Dengue                                     float64
Drownings                                  float64
Fires                                      float64
HIV/AIDS                                   float64
Leukaemia                                  float64
Malaria                                    float64
Natural disasters                          float64
Road traffic accidents                     float64
Self-inflicted injuries                    float64
Tuberculosis                               float64
Violence                                   float64
War                                        float64
dtype: object

Ví dụ một phần của dữ liệu

In [62]:
df.query("Country == 'Italy' and Year == '2015-01-01'") # Dữ liệu Italy năm 2015

,Continent,Country,Year,Age,Brain and nervous system cancers,Covid-19,Dengue,Drownings,Fires,HIV/AIDS,Leukaemia,Malaria,Natural disasters,Road traffic accidents,Self-inflicted injuries,Tuberculosis,Violence,War
6826,Europe,Italy,2015-01-01,0,3.0,0.0,0.0,82.0,134.0,0.0,NaN,0.0,0.0,7.0,0.0,0.0,2.0,0.0
6827,Europe,Italy,2015-01-01,1-4,13.0,0.0,0.0,5.0,0.0,0.0,20.0,0.0,0.0,11.0,0.0,1.0,1.0,0.0
6828,Europe,Italy,2015-01-01,15-24,38.0,0.0,0.0,27.0,5.0,3.0,57.0,0.0,7.0,452.0,195.0,1.0,38.0,0.0
6829,Europe,Italy,2015-01-01,25-34,57.0,0.0,0.0,35.0,7.0,29.0,66.0,0.0,6.0,429.0,335.0,7.0,61.0,0.0
6830,Europe,Italy,2015-01-01,35-54,657.0,0.0,0.0,70.0,34.0,401.0,358.0,1.0,33.0,932.0,1383.0,31.0,166.0,0.0
6831,Europe,Italy,2015-01-01,5-14,39.0,0.0,0.0,10.0,0.0,0.0,48.0,0.0,0.0,25.0,13.0,1.0,2.0,0.0
6832,Europe,Italy,2015-01-01,55-74,2011.0,0.0,0.0,120.0,51.0,222.0,1737.0,0.0,15.0,821.0,1237.0,65.0,81.0,0.0
6833,Europe,Italy,2015-01-01,75+,1320.0,0.0,0.0,82.0,134.0,49.0,3745.0,1.0,2.0,750.0,825.0,226.0,44.0,0.0
